In [1]:
from module.prepare import *
from itertools import product
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

import gc
import os
import re
import math
import sys
from collections import Counter
import random
from itertools import islice
import time
import configparser
import json
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import bokeh

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy as sp
from scipy import stats

import sklearn
from joblib import dump, load
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.manifold import *
import sklearn.tree as Tr 

import lightgbm as lgb

from module.metrics import *

D:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [44]:
def getCurrentTime():
    return datetime.datetime.strftime(datetime.datetime.fromtimestamp(time.time()),format='%Y-%m-%d-%H-%M-%S')


def LGBTuning(Xtrain,Xtest,Ytrain,Ytest,new_params=None):
    
    clf = lgb.LGBMClassifier(objective='cross_entropy', ### {cross_entropy, binary}
                             silent=False,
                             verbose=1,
                             random_state=seed,
                             n_jobs=20,
#                              class_weight
                            )
    
    default_params = {
    'learning_rate': [0.1], 
    'boosting_type':['gbdt'], 
    'n_estimators': [500],
    'num_iterations':[1000],
    'max_bin':[256]
    }
    
    if new_params is not None:
        default_params.update(new_params)
    
    arg_str = ''
    for k,v in default_params.items():
        if type(v[0])==str:
            arg_str += k+'='+"'"+v[0]+"',"
        else:
            arg_str += k+'='+str(v[0])+","
    eval(
        'clf.'+clf.set_params.__name__+"("
            +arg_str.rstrip(',')+
            ")"
        )

    print('DEBUG:: tuning params\n',clf.get_params())
    clf.fit(Xtrain,Ytrain)
    Ypred = clf.predict(Xtest)
    score_train = clf.score(Xtrain,Ytrain)
    print('train score %f'%score_train)
    
    return [Ypred,Ytest,score_train,clf]

In [3]:
cv = 5
generalize_ratio = 1.0/cv
test_ratio = 1.0/cv
tuning_mode = False

if tuning_mode:
    cv = 1

tuning 1

In [4]:
def TuningParametersStage1(fname=getCurrentTime()+'-stage1.csv'):
    res = []
    for DATAID in [0,1,2,3,4,5]:
        INFO('data id %d'%DATAID)
        for RNA_K in range(3,7):
            for PROTEIN_K in range(3,7):
                for TOP_RATIO in np.linspace(0.93,0.99,5):
                    start_time = time.time()
                    [data,T] = ReadData(DATAID,PROTEIN_K,RNA_K)
                    [X,Y] = ToMatrix(data,'dense')
                    [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
                    [X_train,X_test,Y_train,Y_test] = \
                                RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=TOP_RATIO)
                    r = LGBTuning(X_train,X_test,Y_train,Y_test)
                    r = {
                        'DATAID': DATAID,
                        'test_score':scoreFunction(r[0],r[1]),
                        'train_score':r[2],
                        'RNA_K':RNA_K,
                        'PROTEIN_K':PROTEIN_K,
                        'TOP_RATIO':TOP_RATIO
                    }
                    print('DEBUG:: result ',r)
                    res.append(r)
                    end_time = time.time()
                    print('DEBUG:: time elapsed ',(end_time-start_time)/60)
    df = pd.DataFrame(data=res,columns=['DATAID','test_score','train_score','RNA_K','PROTEIN_K','TOP_RATIO'])
    df.to_csv(os.path.join('./result',fname))
    
    return


In [ ]:
TuningParametersStage1()

In [31]:
import json
import re

# fname_optimal_stage1 = './result/2020-05-04-20-55-26-stage1.csv'
# df = pd.read_csv(fname_optimal_stage1)
# acc = [ float(re.findall('[0-9]+\.[0-9]+',x)[0]) for x in df['test_score'] ]
# auc = [ float(re.findall('[0-9]+\.[0-9]+',x)[1]) for x in df['test_score'] ]
# df['acc'] = acc
# df['auc'] = auc

# new_df1 = df.groupby(by=['DATAID','PROTEIN_K','RNA_K']).agg({'acc':np.max,'auc':np.max})
# new_df2 = df.groupby(by=['DATAID','PROTEIN_K','RNA_K']).agg({'acc':np.mean,'auc':np.mean})

params_1 = {
    0:[(5,5,0.99)],
    1:[(3,4,0.96),(3,3,0.99)],
    2:[(6,6,0.96),(4,4,0.96)],
    3:[(3,3,0.975),(6,3,0.96)],
    4:[(4,3,0.96)],
    5:[(4,6,0.945)]
}



stage 2

In [38]:
fname_result2 = getCurrentTime()+'-stage2.csv'

tune_grid = [
    [{
        'learning_rate': [0.1], ### 0.1
        'boosting_type':['gbdt'], ### goss>gbdt
        'n_estimators': [500],
        'num_iterations':[1000,2000], ### 2000
        'num_leaves': [200], ### <400<675
        'max_bin':[256],
        'colsample_bytree' : [0.7,0.75,0.8,0.85], ### 0.75
        'subsample_freq':[1], ### 1
        'lambda_l1': [1e-3,0]
    }],
    [{
        'colsample_bytree': [1], 
        'lambda_l1': [0], 
        'learning_rate': [0.05], 
        'min_data_in_leaf': [12,13,14,15], 
        'num_iterations': [1000], 
        'num_leaves': [80,100,120,150,200]
    }],
    [{
        'colsample_bytree': [0.7,0.8,0.85,0.9,0.95], 
        'lambda_l1': [0], # =
        'learning_rate': [0.05,0.01,0.1], 
        'min_data_in_leaf': [20], # =
        'num_iterations': [500,1000,2000],
        'num_leaves': [50]
     }],
    [{
        'colsample_bytree': [0.8,0.85,0.9,0.95],  
        'lambda_l1': [0.001], 
        'learning_rate': [0.1,0.001,0.5],  
        'max_depth': [3,4,6,-1], 
        'min_data_in_leaf': [10,18,30], 
        'num_iterations': [500,1000,2000], 
        'num_leaves': [10,40,60,80,100,120]
     }],
    [{
        'colsample_bytree': [0.8,0.85,0.9,0.95],  
        'lambda_l1': [0], 
        'learning_rate': [0.01,0.001,0.5], 
        'max_depth': [3,4,6,-1],  
        'min_data_in_leaf': [10,18,30], 
        'num_iterations': [500,1000,2000], 
        'num_leaves': [10,40,60,80,100,120]
     }],
    [{
        'colsample_bytree': [0.8,0.85,0.9,0.95],  
        'lambda_l1': [0], 
        'learning_rate': [0.01,0.001,0.5], 
        'max_depth': [3,4,6,-1],  
        'min_data_in_leaf': [10,18,30], 
        'num_iterations': [500,1000,2000], 
        'num_leaves': [10,40,60,80,100,120]
     }]
]

tune_grid = list(map(lambda x:list(ParameterGrid(x)),tune_grid))


In [37]:
json.dumps({'1':2})

'{"1": 2}'

In [ ]:
tuning_cv2 = 1
tuning_generalize_ratio2 = 1.0/tuning_cv2 if tuning_cv2!=1 else 0.2

df_columns = ['dataid','protein_k','rna_k','top_ratio','training_score','tune_param','scores']
df_result2 = pd.DataFrame([],columns=df_columns)

for _dataid in [0,1,2,3,4,5]:
    INFO('dataid %d'%_dataid)
    for global_params in params_1[_dataid]:
        ### get conf of dataid
        protein_k = global_params[0]
        rna_k = global_params[1]
        top_ratio = global_params[2]
        ### read data
        [data,T] = ReadData(_dataid,protein_k,rna_k)
        [X,Y] = ToMatrix(data,'dense')
        ### split dataset
        [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,tuning_generalize_ratio2)
        ### dimensionality reduction
        [X_train,X_test,Y_train,Y_test] = \
                    RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=top_ratio)
        for _cv in range(tuning_cv2):
            INFO('tuning cv %d'%_cv)
            for sp in tune_grid[_dataid]:
                sp = dict(map(lambda x:(x,[sp[x]]),sp))
                print(sp)
                tune_results = LGBTuning(X_train,X_test,Y_train,Y_test,sp)
                tune_score = scoreFunction(tune_results[0],tune_results[1])
                r = pd.Series({
                                'dataid':_dataid,
                                'protein_k':protein_k,
                                'rna_k':rna_k,
                                'top_ratio':top_ratio,
                                'training_score':tune_results[2],
                                'tune_param':json.dumps(sp),
                                'scores':json.dumps(tune_score),
                })
                print(r)
                break 
                df_result2 = df_result2.append(r,ignore_index=True)
            
df_result2.to_csv(fname_result2)
            

INFO::dataid 0
read data 5 5
=============RETRIEVE TRIAN DATA=================
# DEBUG: # DEBUG: **************new dl 0***************
# DEBUG: READ SEQ FROM FILE
# DEBUG: READ CLUSTER FROM FILE
ERROR:: regex  
ERROR:: regex  
# DEBUG: READ PAIR FROM FILE
read line error 
# DEBUG: GENERATE NEGATIVE PAIR
# DEBUG: negative pair number 10411
INFO::count of negative pairs10411
# DEBUG: PAIR UNION
# DEBUG: EXTRACT FEATURES--PROTEIN
# DEBUG: EXTRACT FEATURES--RNA
# DEBUG: K-MER CALCULATION
# DEBUG: FEATURE UNION
# DEBUG: GARBAGE COLLECTION
MATRIX TRANSFORMATION
DEBUG:: total features count  32068
data shape 20822 32068
rf raw data fit score 1.000000
INFO::dimension remained 31907 0.990000
dimension remained 31907
INFO::tuning cv 0
{'boosting_type': ['gbdt'], 'colsample_bytree': [0.7], 'lambda_l1': [0.001], 'learning_rate': [0.1], 'max_bin': [256], 'n_estimators': [500], 'num_iterations': [1000], 'num_leaves': [200], 'subsample_freq': [1]}
DEBUG:: tuning params
 {'boosting_type': 'gbdt', 'cla

D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
